# Purpose of this notebook

The goal of this notebook is to pull all the raw CSV files for this project from the directory on AWS. This is the original notebook that got me the datasets.

This code is re-used in the master_df_cleaning notebook but this is the original notebook.

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import re

# Move to the data folder for this project
os.chdir('C:/Users/riley/Documents/Coding/DSC/datas/baywheels-data/')

# Get the main URL and turn it into XML soup
r = requests.get("https://s3.amazonaws.com/baywheels-data/")
data = r.text
soup = BeautifulSoup(data, "xml")

# The data file URLs are in the <Key> node so find that
keys = soup.find_all('Key')

key_list = []

# Grab the name of each file by taking the text attribute of the key node
for key in keys:
    key_list.append(key.text)
    
# Slice off the trailing file, index.html
key_list_final = key_list[:-1]

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [13]:
# Downloads all files in the list of filenames.
# Deal with extracting them in the next cell.

import zipfile, urllib.request, shutil

for key in key_list_final:

    url = 'https://s3.amazonaws.com/baywheels-data/' + key
    file_name = key

    with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

# Unzip locally

In [14]:
# For all items ending in .zip in the dir_name, 
# extract their contents to dir_name and delete the original zip folder.

extension = ".zip"
dir_name = 'C:/Users/riley/Documents/Coding/DSC/datas/baywheels-data'

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [15]:
# Create a list of all CSVs in the directory to iterate over and make
# dataframes from.

dir_name = 'C:/Users/riley/Documents/Coding/DSC/datas/baywheels-data/'

# List of all .csv filenames to be read into dataframes
csv_list = os.listdir(dir_name)

# Build a list of dataframe names by removing .csv
df_names = []

for csv in csv_list:
    df_name = csv.replace('.csv', '')
    df_names.append(df_name)

In [16]:
# Names of DFs and the filenames
df_dict_names = dict(zip(df_names, csv_list))

# Create the master dataframe

In [17]:
import pandas as pd

# Create a dictionary of dataframes to then concatenate into one dataframe

dict_of_dataframes = {}
for i in range(len(df_names)):
    dict_of_dataframes[df_names[i]] = pd.read_csv(csv_list[i])

In [18]:
master_df = pd.concat(dict_of_dataframes.values())

c:\users\riley\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
